In [1]:
#imports

import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
#data
from dataset import ONRData

In [2]:
ONR = ONRData(label_dtype='int', feature_extraction=True)

data = ONR.X
labels = ONR.y
groups = ONR.groups

#convert to np
data = np.array(data)
labels = np.array(labels)
groups = np.array(groups)



sample shape:(15, 178)
sample shape after feature extraction:(432,)
sample shape:(15, 178)
sample shape after feature extraction:(432,)
sample shape:(15, 178)
sample shape after feature extraction:(432,)
sample shape:(15, 178)
sample shape after feature extraction:(432,)
sample shape:(15, 178)
sample shape after feature extraction:(432,)
sample shape:(15, 178)
sample shape after feature extraction:(432,)
sample shape:(15, 178)
sample shape after feature extraction:(432,)
sample shape:(15, 178)
sample shape after feature extraction:(432,)
sample shape:(15, 178)
sample shape after feature extraction:(432,)
sample shape:(15, 178)
sample shape after feature extraction:(432,)
sample shape:(15, 178)
sample shape after feature extraction:(432,)
sample shape:(15, 178)
sample shape after feature extraction:(432,)
sample shape:(15, 178)
sample shape after feature extraction:(432,)
sample shape:(15, 178)
sample shape after feature extraction:(432,)
sample shape:(15, 178)
sample shape after featur

In [3]:
print("Data shape: ", data.shape)
print("Labels shape: ", labels.shape)
print("Groups shape: ", groups.shape)


Data shape:  (4373, 432)
Labels shape:  (4373,)
Groups shape:  (4373,)


In [4]:
from sklearn.model_selection import StratifiedGroupKFold
import numpy as np

# Initialize StratifiedGroupKFold with 5 splits
skf = StratifiedGroupKFold(n_splits=5, shuffle=True, random_state=42)

for train_index, test_index in skf.split(data, labels, groups):
    X_train, X_test = data[train_index], data[test_index]
    y_train, y_test = labels[train_index], labels[test_index]
    groups_train, groups_test = groups[train_index], groups[test_index]

print("Train shape: ", X_train.shape)
print("Test shape: ", X_test.shape)
print("Train labels shape: ", y_train.shape)
print("Test labels shape: ", y_test.shape)
print("Train groups shape: ", groups_train.shape)
print("Test groups shape: ", groups_test.shape)


Train shape:  (3617, 432)
Test shape:  (756, 432)
Train labels shape:  (3617,)
Test labels shape:  (756,)
Train groups shape:  (3617,)
Test groups shape:  (756,)


In [6]:
#perform smote on the train data
from imblearn.over_sampling import SMOTE
smote = SMOTE(sampling_strategy='auto', random_state=42)
original_shape = X_train.shape
print(f"Original shape: {original_shape}")

# Reshape X_train for SMOTE (flatten the data if needed)
X_train_reshaped = X_train.reshape(X_train.shape[0], -1)
print(f"Reshaped for SMOTE: {X_train_reshaped.shape}")

# Apply SMOTE
X_resampled_2d, y_resampled = smote.fit_resample(X_train_reshaped, y_train)

# # Reshape the oversampled data back to 3D
# X_resampled = X_resampled_2d.reshape(-1, original_shape[1], original_shape[2], original_shape[3])
X_resampled = X_resampled_2d
print(f"Reshaped back to 3D: {X_resampled.shape}")

Original shape: (3617, 432)
Reshaped for SMOTE: (3617, 432)
Reshaped back to 3D: (6028, 432)


In [8]:
# Check class distribution after SMOTE
from collections import Counter
print("Class distribution after SMOTE:", Counter(y_resampled))

Class distribution after SMOTE: Counter({np.int64(0): 3014, np.int64(1): 3014})


In [17]:

#reshape for random forest
X_train_reshaped = X_resampled.reshape(X_resampled.shape[0], -1)
X_test_reshaped = X_test.reshape(X_test.shape[0], -1)

from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_estimators=10, random_state=42, class_weight={0:1, 1:1.5}, n_jobs=-1, max_depth=10, min_samples_split=2, min_samples_leaf=1, max_features='sqrt')
rf.fit(X_train_reshaped, y_resampled)
#predict
y_pred = rf.predict(X_test_reshaped)
from sklearn.metrics import classification_report, confusion_matrix
print("Classification report: ")
print(classification_report(y_test, y_pred))
print("Confusion matrix: ")
print(confusion_matrix(y_test, y_pred))

train_pred = rf.predict(X_train_reshaped)
print("Train classification report: ")
print(classification_report(y_resampled, train_pred))
print("Train confusion matrix: ")
print(confusion_matrix(y_resampled, train_pred))


Classification report: 
              precision    recall  f1-score   support

           0       0.82      0.51      0.63       634
           1       0.14      0.43      0.21       122

    accuracy                           0.49       756
   macro avg       0.48      0.47      0.42       756
weighted avg       0.71      0.49      0.56       756

Confusion matrix: 
[[321 313]
 [ 70  52]]
Train classification report: 
              precision    recall  f1-score   support

           0       0.99      0.83      0.91      3014
           1       0.86      1.00      0.92      3014

    accuracy                           0.91      6028
   macro avg       0.93      0.91      0.91      6028
weighted avg       0.93      0.91      0.91      6028

Train confusion matrix: 
[[2511  503]
 [  14 3000]]


In [ ]:
#grid search for parameters
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import make_scorer, f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score, recall_score

# Define the parameter grid
param_grid = {
    'n_estimators': [10, 50, 100],
    'max_depth': [None,5, 10, 20],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2],
    'max_features': ['sqrt', 'log2'],
    'bootstrap': [True, False],
    'class_weight': ['balanced', None]
    
}

# Initialize the RandomForestClassifier
rf = RandomForestClassifier(random_state=42, class_weight='balanced')
# Initialize GridSearchCV with f1_score as the scoring metric
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, scoring='f1_macro', cv=3, n_jobs=-1)
# Fit the grid search to the training data
grid_search.fit(X_train_reshaped, y_resampled)
# Get the best parameters and score
best_params = grid_search.best_params_
best_score = grid_search.best_score_
print("Best parameters: ", best_params)
print("Best score: ", best_score)
#predict
y_pred = grid_search.predict(X_test_reshaped)
print("Classification report: ")
print(classification_report(y_test, y_pred))
print("Confusion matrix: ")
print(confusion_matrix(y_test, y_pred))


Best parameters:  {'bootstrap': False, 'class_weight': 'balanced', 'max_depth': None, 'max_features': 'log2', 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 10}
Best score:  0.6214119103413641
Classification report: 
              precision    recall  f1-score   support

           0       0.84      0.92      0.88       634
           1       0.22      0.11      0.15       122

    accuracy                           0.79       756
   macro avg       0.53      0.52      0.52       756
weighted avg       0.74      0.79      0.76       756

Confusion matrix: 
[[585  49]
 [108  14]]


In [18]:
#SVM
from sklearn.svm import SVC

from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score, recall_score

# Define the parameter grid
param_grid = {
    'C': [0.1, 1, 10],
    'kernel': ['linear', 'rbf'],
    'gamma': ['scale', 'auto'],
    'class_weight': ['balanced', None]
}

# Initialize the SVC
svm = SVC(random_state=42)
# Initialize GridSearchCV with f1_score as the scoring metric
grid_search = GridSearchCV(estimator=svm, param_grid=param_grid, scoring='f1_macro', cv=3, n_jobs=-1)
# Fit the grid search to the training data
grid_search.fit(X_train_reshaped, y_resampled)
# Get the best parameters and score
best_params = grid_search.best_params_
best_score = grid_search.best_score_
print("Best parameters: ", best_params)

print("Best score: ", best_score)

#predict
y_pred = grid_search.predict(X_test_reshaped)
print("Classification report: ")
print(classification_report(y_test, y_pred))
print("Confusion matrix: ")
print(confusion_matrix(y_test, y_pred))
#predict on train data
train_pred = grid_search.predict(X_train_reshaped)
print("Train classification report: ")
print(classification_report(y_resampled, train_pred))
print("Train confusion matrix: ")
print(confusion_matrix(y_resampled, train_pred))


KeyboardInterrupt: 